<a href="https://colab.research.google.com/github/sulaimanbehzad/Clustering_according_to_item_popularity/blob/main/price_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimating the price of each product
We will use Neural Network to predict the price of each product

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, normalize, MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso 
from sklearn.linear_model import BayesianRidge 
from sklearn.linear_model import ElasticNet 

from sklearn.metrics import explained_variance_score as evs
from sklearn.metrics import r2_score as r2
from termcolor import colored as cl
from sklearn import metrics

In [53]:
products = pd.read_excel('/content/drive/MyDrive/Clustering_according_to_item_popularity/product.xlsx')


BadZipFile: ignored

In [48]:
products.head()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


For the above purpose we will need these features:  
1. ID_Item
2. Quantity_item
3. Amount_Gross Order


In [13]:
orders = orders.filter(['ID_Item', 'Quantity_item', 'Amount_Gross_Order'])
orders.head()

,ID_Item,Quantity_item,Amount_Gross_Order
0,21386,1.0,597982.0
1,248497,1.0,980000.0
2,50144,1.0,229358.0
3,70208,1.0,16514.0
4,67627,1.0,133028.0


In [14]:
orders['Unit_Price'] = np.divide(orders['Amount_Gross_Order'], orders['Quantity_item'])
orders['Unit_Price']

0         597982.0
1         980000.0
2         229358.0
3          16514.0
4         133028.0
            ...   
199995    172018.5
199996    600000.0
199997    450000.0
199998    193486.0
199999     55046.0
Name: Unit_Price, Length: 200000, dtype: float64

In [15]:
orders.head(30)

,ID_Item,Quantity_item,Amount_Gross_Order,Unit_Price
0,21386,1.0,597982.0,5.979820e+05
1,248497,1.0,980000.0,9.800000e+05
2,50144,1.0,229358.0,2.293580e+05
3,70208,1.0,16514.0,1.651400e+04
4,67627,1.0,133028.0,1.330280e+05
5,312932,1.0,290000.0,2.900000e+05
6,42365,1.0,55046.0,5.504600e+04
7,715680,1.0,2790000.0,2.790000e+06
8,102588,1.0,34862.0,3.486200e+04
9,658519,1.0,2600000.0,2.600000e+06


Let's inspect the database

In [16]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID_Item             200000 non-null  int64  
 1   Quantity_item       200000 non-null  float64
 2   Amount_Gross_Order  200000 non-null  float64
 3   Unit_Price          200000 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 6.1 MB


In [17]:
orders.isnull().sum()

ID_Item               0
Quantity_item         0
Amount_Gross_Order    0
Unit_Price            0
dtype: int64

In [22]:
X = orders['ID_Item'].values
X

array([ 21386, 248497,  50144, ..., 215045, 264021,  89427])

In [24]:
Y = orders['Unit_Price'].values
Y

array([597982., 980000., 229358., ..., 450000., 193486.,  55046.])

No null values, good to go



In [26]:
min_max_scaler = MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X.reshape(-1,1))
X_scale

array([[0.01017842],
       [0.11865473],
       [0.02391426],
       ...,
       [0.10267686],
       [0.12606955],
       [0.04267722]])

In [27]:
x_train,x_test,y_train,y_test=train_test_split(X_scale,Y,test_size=0.2,random_state=123)

In [29]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(160000, 1) (40000, 1) (160000,) (40000,)


In [31]:
# 1. OLS

ols = LinearRegression()
ols.fit(x_train, y_train)
ols_yhat = ols.predict(x_test)

# 2. Ridge

ridge = Ridge(alpha = 0.5)
ridge.fit(x_train, y_train)
ridge_yhat = ridge.predict(x_test)

# 3. Lasso

lasso = Lasso(alpha = 0.01)
lasso.fit(x_train, y_train)
lasso_yhat = lasso.predict(x_test)

# 4. Bayesian

bayesian = BayesianRidge()
bayesian.fit(x_train, y_train)
bayesian_yhat = bayesian.predict(x_test)

# 5. ElasticNet

en = ElasticNet(alpha = 0.01)
en.fit(x_train, y_train)
en_yhat = en.predict(x_test)

In [44]:
print(cl('EXPLAINED VARIANCE SCORE:', attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('Explained Variance Score of OLS model is {}'.format(evs(y_test, ols_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('Explained Variance Score of Ridge model is {}'.format(evs(y_test, ridge_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('Explained Variance Score of Lasso model is {}'.format(evs(y_test, lasso_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('Explained Variance Score of Bayesian model is {}'.format(evs(y_test, bayesian_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('Explained Variance Score of ElasticNet is {}'.format(evs(y_test, en_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')

EXPLAINED VARIANCE SCORE:
-------------------------------------------------------------------------------
Explained Variance Score of OLS model is 0.0
-------------------------------------------------------------------------------
Explained Variance Score of Ridge model is 0.0
-------------------------------------------------------------------------------
Explained Variance Score of Lasso model is 0.0
-------------------------------------------------------------------------------
Explained Variance Score of Bayesian model is 0.0
-------------------------------------------------------------------------------
Explained Variance Score of ElasticNet is 0.0
-------------------------------------------------------------------------------


In [45]:
print(cl('R-SQUARED:', attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('R-Squared of OLS model is {}'.format(r2(y_test, ols_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('R-Squared of Ridge model is {}'.format(r2(y_test, ridge_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('R-Squared of Lasso model is {}'.format(r2(y_test, lasso_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('R-Squared of Bayesian model is {}'.format(r2(y_test, bayesian_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('R-Squared of ElasticNet is {}'.format(r2(y_test, en_yhat).round()), attrs = ['bold']))
print('-------------------------------------------------------------------------------')

R-SQUARED:
-------------------------------------------------------------------------------
R-Squared of OLS model is -0.0
-------------------------------------------------------------------------------
R-Squared of Ridge model is -0.0
-------------------------------------------------------------------------------
R-Squared of Lasso model is -0.0
-------------------------------------------------------------------------------
R-Squared of Bayesian model is -0.0
-------------------------------------------------------------------------------
R-Squared of ElasticNet is -0.0
-------------------------------------------------------------------------------
